# Shubham Sharma
## IIT BOMBAY


This is a basic code for contractive autoencoder.

In [0]:
from __future__ import division, print_function, absolute_import

import tensorflow as tf
import matplotlib.pyplot as plt

# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)


W0703 10:12:23.802644 140384207398784 deprecation.py:323] From <ipython-input-1-af7bdde62763>:8: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0703 10:12:23.807456 140384207398784 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0703 10:12:23.810437 140384207398784 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:262: extract_images (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instruction

Extracting /tmp/data/train-images-idx3-ubyte.gz


W0703 10:12:24.097955 140384207398784 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:267: extract_labels (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.
W0703 10:12:24.101504 140384207398784 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:110: dense_to_one_hot (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.one_hot on tensors.
W0703 10:12:24.160852 140384207398784 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be remove

Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [0]:
# Training Parameters
learning_rate = 0.001
num_steps = 10000
#num_steps=50000
batch_size = 200

display_step = 500
examples_to_show = 10

# Network Parameters
num_hidden_1 = 20 #200 # 1st layer num features
num_hidden_2 = 20 #90 # 2nd layer num features (the latent dim)
num_input = 784 # MNIST data input (img shape: 28*28)


In [0]:
# tf Graph input (only pictures)
X = tf.placeholder("float", [None, num_input])

weights = {
    'encoder_h1': tf.Variable(tf.random_normal([num_input, num_hidden_1])),
    'encoder_h2': tf.Variable(tf.random_normal([num_hidden_1, num_hidden_2])),
    'decoder_h1': tf.Variable(tf.random_normal([num_hidden_2, num_hidden_1])),
    'decoder_h2': tf.Variable(tf.random_normal([num_hidden_1, num_input])),
}
biases = {
    'encoder_b1': tf.Variable(tf.random_normal([num_hidden_1])),
    'encoder_b2': tf.Variable(tf.random_normal([num_hidden_2])),
    'decoder_b1': tf.Variable(tf.random_normal([num_hidden_1])),
    'decoder_b2': tf.Variable(tf.random_normal([num_input])),
}


In [0]:
# Building the encoder
def encoder(x):
    # Encoder Hidden layer with sigmoid activation #1
    layer_1 = (tf.add(tf.matmul(x, weights['encoder_h1']),
                                   biases['encoder_b1']))
    # Encoder Hidden layer with sigmoid activation #2
    layer_2 = (tf.add(tf.matmul(layer_1, weights['encoder_h2']),
                                   biases['encoder_b2']))
    return layer_2


# Building the decoder
def decoder(x):
    # Decoder Hidden layer with sigmoid activation #1
    layer_1 = (tf.add(tf.matmul(x, weights['decoder_h1']),
                                   biases['decoder_b1']))
    # Decoder Hidden layer with sigmoid activation #2
    layer_2 = (tf.add(tf.matmul(layer_1, weights['decoder_h2']),
                                   biases['decoder_b2']))
    return layer_2


In [0]:
# Construct model
encoder_op = encoder(X)
decoder_op = decoder(encoder_op)

# Prediction
y_pred = decoder_op
# Targets (Labels) are the input data.
y_true = X

from tensorflow.python.ops.parallel_for.gradients import jacobian
J=jacobian(encoder_op,X)

# Define loss and optimizer, minimize the squared error
loss = tf.reduce_mean(tf.pow(y_true - y_pred, 2))
regularized_term = tf.norm(J)
#regularized_term = 0


contractive_loss = regularized_term + loss
optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(contractive_loss)


W0703 10:12:24.694988 140384207398784 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1205: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0703 10:12:24.817616 140384207398784 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/rmsprop.py:119: calling Ones.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [0]:
# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()
loss_list=[]
# Start Training
# Start a new TF session
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    # Training
    for i in range(1, num_steps+1):
        # Prepare Data
        # Get the next batch of MNIST data (only images are needed, not labels)
        batch_x, _ = mnist.train.next_batch(batch_size)

        # Run optimization op (backprop) and cost op (to get loss value)
        _, l = sess.run([optimizer, contractive_loss], feed_dict={X: batch_x})
        # Display logs per step
        if i % display_step == 0 or i == 1:
          print('Step %i: Minibatch Loss: %f' % (i, l))


Step 1: Minibatch Loss: 574909.750000
Step 500: Minibatch Loss: 6500.844727
Step 1000: Minibatch Loss: 1466.479492
Step 1500: Minibatch Loss: 54.894550
Step 2000: Minibatch Loss: 8.872515
Step 2500: Minibatch Loss: 13.720474
Step 3000: Minibatch Loss: 13.482314
Step 3500: Minibatch Loss: 14.039104
Step 4000: Minibatch Loss: 13.863711
Step 4500: Minibatch Loss: 13.860048
Step 5000: Minibatch Loss: 13.632374
Step 5500: Minibatch Loss: 13.401568
Step 6000: Minibatch Loss: 13.897683
Step 6500: Minibatch Loss: 13.669905
Step 7000: Minibatch Loss: 13.721952
Step 7500: Minibatch Loss: 13.787473
Step 8000: Minibatch Loss: 13.431569
Step 8500: Minibatch Loss: 13.598118
Step 9000: Minibatch Loss: 13.980861
Step 9500: Minibatch Loss: 13.733674
Step 10000: Minibatch Loss: 13.585894
